In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, HistGradientBoostingRegressor, VotingRegressor, StackingRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
train=pd.read_csv('train_full.csv',index_col=[0])
train.shape

(1458, 173)

In [3]:
train.head()

,MSSubClass,LotFrontage,LotArea,LotShape,LandContour,LandSlope,BldgType,HouseStyle,OverallQual,OverallCond,...,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd
0,10,65.0,8450,3,3,0,0,5,7,5,...,0,0,0,0,0,1,0,0,0,0
1,5,80.0,9600,3,3,0,0,2,6,8,...,0,0,0,0,0,1,0,0,0,0
2,10,68.0,11250,0,3,0,0,5,7,5,...,0,0,0,0,0,1,0,0,0,0
3,11,60.0,9550,0,3,0,0,5,7,5,...,0,0,0,0,0,0,0,0,0,1
4,10,84.0,14260,0,3,0,0,5,8,5,...,0,0,0,0,0,1,0,0,0,0


In [22]:
y=train['SalePrice']
x=train.drop(['SalePrice'],axis=1)
X_train, X_valid, y_train, y_valid=train_test_split(x,y,random_state=73)

In [5]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, x.values, y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [6]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [7]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.1135 (0.0060)



In [8]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.1135 (0.0061)



In [9]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\Users\CHOWKELVIN\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:188: LinAlgWarning: Ill-conditioned matrix (rcond=2.73212e-19): result may not be accurate.
  overwrite_a=False)
D:\Users\CHOWKELVIN\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:188: LinAlgWarning: Ill-conditioned matrix (rcond=6.31053e-19): result may not be accurate.
  overwrite_a=False)
D:\Users\CHOWKELVIN\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:188: LinAlgWarning: Ill-conditioned matrix (rcond=2.85448e-19): result may not be accurate.
  overwrite_a=False)
D:\Users\CHOWKELVIN\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:188: LinAlgWarning: Ill-conditioned matrix (rcond=2.38671e-19): result may not be accurate.
  overwrite_a=False)


Kernel Ridge score: 1.6932 (1.0302)



D:\Users\CHOWKELVIN\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:188: LinAlgWarning: Ill-conditioned matrix (rcond=2.45003e-19): result may not be accurate.
  overwrite_a=False)


In [10]:
def root_mean_squared_log_error(y_valid, y_preds):
    """Calculate root mean squared error of log(y_true) and log(y_pred)"""
    if len(y_preds)!=len(y_valid): return 'error_mismatch'
    y_preds_new = [math.log(x) for x in y_preds]
    y_valid_new = [math.log(x) for x in y_valid]
    return mean_squared_error(y_valid_new, y_preds_new, squared=False)

In [23]:
RF_f=RandomForestRegressor(bootstrap=False, max_depth=60, max_features='sqrt',
                      min_samples_split=4, n_estimators=1700)
RF_f.fit(X_train,y_train)
y_pred_RF_f=RF_f.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_RF_f))
#0.141125

RMSLE: 0.1359466044233098


In [24]:
XT=ExtraTreesRegressor(n_estimators=1000,random_state=73)
XT.fit(X_train,y_train)
y_pred_XT=XT.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_XT))
#0.140768

RMSLE: 0.132730055900028


In [25]:
Ada_f=AdaBoostRegressor(learning_rate=0.03, loss='exponential', n_estimators=2300,
                  random_state=73)
Ada_f.fit(X_train,y_train)
y_pred_Ada_f=Ada_f.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_Ada_f))
#0.189845

RMSLE: 0.1840811376048676


In [26]:
GB_f=GradientBoostingRegressor(max_depth=10, max_features='sqrt',
                          min_samples_split=12, n_estimators=1500)
GB_f.fit(X_train,y_train)
y_pred_GB_f=GB_f.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_GB_f))
#0.136772

RMSLE: 0.1301974304192531


In [27]:
HGB_f=HistGradientBoostingRegressor(learning_rate=0.02,
                              loss='least_absolute_deviation', max_depth=40,
                              max_iter=750, min_samples_leaf=2,
                              random_state=73)
HGB_f.fit(X_train,y_train)
y_pred_HGB_f=HGB_f.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_HGB_f))
#0.132428

RMSLE: 0.12826726693601748


In [28]:
XGB_f=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0.5, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=4,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=1200, n_jobs=0, num_parallel_tree=1, random_state=73,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1.0,
             tree_method='exact', validate_parameters=1, verbosity=None)
XGB_f.fit(X_train,y_train)
y_pred_XGB_f=XGB_f.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_XGB_f))
#0.120656

RMSLE: 0.12196873766102913


In [29]:
LGBg_f=LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
LGBg_f.fit(X_train,y_train)
y_pred_LGBg_f=LGBg_f.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_LGBg_f))
#0.123494

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
RMSLE: 0.12304024055396307


In [30]:
Vote=VotingRegressor([('gb',GB_f),('hgb',HGB_f),('xgb',XGB_f),('lgb',LGBg_f)])
Vote.fit(X_train,y_train)
y_pred_Vote=Vote.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_Vote))
#0.122400

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
RMSLE: 0.12016821115742808


In [31]:
from sklearn.ensemble import StackingRegressor
estimators=[('gb',GB_f),('hgb',HGB_f),('xgb',XGB_f),('lgb',LGBg_f)]
Stack=StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=800,random_state=42))
Stack.fit(X_train,y_train)
y_pred_Stack=Stack.predict(X_valid)
print('RMSLE:', root_mean_squared_log_error(y_valid, y_pred_Stack))

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11